# M3 Force of Infection ($\bf{\lambda}$)

Force of infection from saved M3 simulations, calculating four "forces" for six models.

The four forces:
 - Live Boar
 - Live Sow
 - Corpse Boar
 - Corpse Sow
 
 
The six models are three networks and two contact functions. 

Networks:
 - Random
 - Scale Free
 - Small World
 
Contact Functions:
 - $C = 1 \quad (C_1)$
 - $C \propto \sqrt{\rho_N} \quad (C_4)$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
#plotting preamble
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 10
})
plt.style.use('seaborn-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [2]:
#path to data
base = "../tau_force/"

In [ ]:
#loading data, the data  has four forces of infection (alive, coprse) X (boar, sow)
#one value per simulation (each run for 500 sims)
sf1 = np.genfromtxt(base+"sf1n.csv",delimiter=",")
sf4 = np.genfromtxt(base+"sf4n.csv",delimiter=",")

sw1 = np.genfromtxt(base+"sw1n.csv",delimiter=",")
sw4 = np.genfromtxt(base+"sw4n.csv",delimiter=",")

rn1 = np.genfromtxt(base+"rn1n.csv",delimiter=",")
rn4 = np.genfromtxt(base+"rn4n.csv",delimiter=",")

In [ ]:
#function to remove runs were ASF dies out, and negative runs 
#(not sure why this occurs, one in every 500 or so is negative)
def filter_out(data, neg = False):
    
    #non-endemic runs set to 9999
    data = data[data[:,0] != 9999]
    
    if not neg : 
        data = data[data[:,4] > 0 ]
    else:
        data = np.abs(data)
    
    return data

In [ ]:
#function to print out output
def print_values(data, neg = False):
    
    data = filter_out(data, neg)
    
    time = data[1,0]
    
    sorted_data = [data[:,1],data[:,2],data[:,3],data[:,4], data[:,1]+data[:,2]+data[:,3]+data[:,4]]
    names = ['Live Boar', 'Live Sow', 'Dead Boar', 'Dead sow', "Sum"]
    
    print("-----------------------")
    print("Length = ", data.shape[0])
    print("-----------------------")
    for i, v in enumerate(sorted_data):
        #v = 365*v/time
        print(names[i])
       
        
        aa = np.round(stats.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=stats.sem(v)),3)
        #weird print is for a latex table
        print(r"$ {} \; \left[{}-{}\right]$".format(np.round(np.mean(v),3),aa[0],aa[1]))
        
        print("-----------------------")
    
    #the total per day (just to get an idea if correct order of magnitude)
    print(np.mean(sorted_data[4]/time))
    
    

In [ ]:
def store_values(data, neg = False):
    
    data = filter_out(data, neg)
    
    time = data[1,0]
    
    sorted_data = [data[:,1],data[:,2],data[:,3],data[:,4], data[:,1]+data[:,2]+data[:,3]+data[:,4]]
    
    store_mean = np.zeros(5)
    store_95   = np.zeros((5,2))
    
    for i, v in enumerate(sorted_data):
        #v = 365*v/time
        
        store_mean[i] = np.mean(v)
        store_95[i,:] = stats.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=stats.sem(v))
        
    
    return store_mean, store_95
    
    

In [ ]:
#storing filtered values for plotting!
sf1 = store_values(sf1)
sf4 = store_values(sf4)


sw1 = store_values(sw1)
sw4 = store_values(sw4)

rn1 = store_values(rn1)
rn4 = store_values(rn4)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,2))
fig.suptitle('Force of infection during endemic transmision', y = 1.075)
labels = ['Live Boars', 'Live Sows', 'Corpse Boars', 'Corpse Sows']
X = np.arange(4)


ax1.bar(X + 0.00, rn1[0][0:-1], color = colors[0], width = 0.25,edgecolor='black', label = "ER")
ax1.bar(X + 0.25, sf1[0][0:-1], color = colors[0], width = 0.25,hatch='////', edgecolor='black', label = "SF")
ax1.bar(X + 0.50, sw1[0][0:-1], color =colors[0], width = 0.25, hatch = 'xxxx',edgecolor='black', label = "SW")
ax1.set_xticks(X+0.25, labels, rotation=45)
ax1.set_ylabel(r' Force of nfection $(\mathbf{\lambda}$)')
ax1.set_title(r"$C = 1$")
ax1.legend(fontsize = 7)

ax2.bar(X + 0.00, rn4[0][0:-1], color = colors[3], width = 0.25,edgecolor='black', label = "ER")
ax2.bar(X + 0.25, sf4[0][0:-1], color = colors[3], width = 0.25,hatch='////', edgecolor='black', label = "SF")
ax2.bar(X + 0.50, sw4[0][0:-1], color =colors[3], width = 0.25, hatch = 'xxxx',edgecolor='black', label = "SW")
ax2.set_xticks(X+0.25, labels, rotation=45)
ax2.set_title(r"$C \propto \sqrt{\rho_N}$")
ax2.legend(fontsize = 7)

ax1.set_ylim([0,0.3])
ax2.set_ylim([0,0.3])

#plt.savefig('forcem3.pdf', format='pdf', bbox_inches='tight')

# M1 and M2 Force!

In [1]:
def run_simulationsm3(path,number,n_sims, save_path, median = False, distance = False):
    
    posteriors = read_inputs(path,number)
    
    n_params = len(posteriors) #two or three fitted parameters!
    
    
    dd = 0
    n_samples = 100000
    
    dp1 = posteriors[0].resample(n_samples)[0]
    dp2 = posteriors[1].resample(n_samples)[0]

    rd.shuffle(dp1)
    rd.shuffle(dp2)
    
    if n_params == 3:
        dp3 = posteriors[2].resample(n_samples)[0]
        rd.shuffle(dp3)
        
    if median:
        if n_params == 2:
            p = {"p1":np.median(dp1),"p2":np.median(dp2)}
        elif n_params ==3:
            p = {"p1":np.median(dp1),"p2":np.median(dp2), "p3":np.median(dp3)}
            
    for i in range(n_sims):
        
        if not median:
            if n_params == 2:
                p = {"p1":dp1[i],"p2":dp2[i]}
            elif n_params ==3:
                p = {"p1":dp1[i],"p2":dp2[i], "p3":dp3[i]}

        
        
        if number == 1:
            model1(p, save_path)
            
        elif number == 2:
            model2(p, save_path)
                
        elif number == 3:
             model3(p, save_path)
                
        else:
            model4(p, save_path)
        
    return 0

In [ ]:
def run_simulationsm1m2(path,number,n_sims, median = False, distance = False):
    
    posteriors = read_inputs(path,number)
    
    n_params = len(posteriors) #two or three fitted parameters!
    
    fi = np.zeros(n_sims)
    fc = np.zeros(n_sims)
    
    
    dd = 0
    n_samples = 100000
    
    dp1 = posteriors[0].resample(n_samples)[0]
    dp2 = posteriors[1].resample(n_samples)[0]

    rd.shuffle(dp1)
    rd.shuffle(dp2)
    
    if n_params == 3:
        dp3 = posteriors[2].resample(n_samples)[0]
        rd.shuffle(dp3)
        
    if median:
        if n_params == 2:
            p = {"p1":np.median(dp1),"p2":np.median(dp2)}
        elif n_params ==3:
            p = {"p1":np.median(dp1),"p2":np.median(dp2), "p3":np.median(dp3)}
            
    for i in range(n_sims):
        
        if not median:
            if n_params == 2:
                p = {"p1":dp1[i],"p2":dp2[i]}
            elif n_params ==3:
                p = {"p1":dp1[i],"p2":dp2[i], "p3":dp3[i]}

        
        
        if number == 1:
            out = model1(p)
            
        elif number == 2:
             out = model2(p)
                
        elif number == 3:
             out = model3(p)
                
        else:
             out = model4(p)
        
        fi[i] = out[0]
        fc[i] = out[1]
       
        
    return fi,fc
        
        
        

In [4]:
#data from M1 and M2 (fast to calculate so not saved, also only live and corpse c1 c4)

M1 = [0.6825,0.0307,0.07,0.4414]
M2 = [1.282,0.427,0.083,0.531]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,2))
fig.suptitle('Force of infection during endemic transmision', y = 1.075)
labels = ['Live', 'Corpse']

X=np.arange(2)

ax1.bar(X + 0.00, M1[0::2], color = colors[0], width = 0.4,edgecolor='black', label = r"$C = 1$")
ax1.bar(X + 0.4, M1[1::2], color = colors[3], width = 0.4, edgecolor='black', label = r"$C \propto \sqrt{\rho_N}$")
ax1.set_xticks(X+0.2, labels, rotation=45)
ax1.set_ylabel(r' Force of nfection $(\mathbf{\lambda}$)')
ax1.set_title(r"M1")

ax2.bar(X + 0.00, M2[0::2], color = colors[0], width = 0.4,edgecolor='black', label = r"$C = 1$")
ax2.bar(X + 0.4, M2[1::2], color = colors[3], width = 0.4, edgecolor='black', label = r"$C \propto \sqrt{\rho_N}$")
ax2.set_xticks(X+0.2, labels, rotation=45)
ax2.set_title(r"M2")
ax2.legend(fontsize = 7)

ax1.set_ylim([0,1.5])
ax2.set_ylim([0,1.5])

#plt.savefig('forcem1m2.pdf', format='pdf', bbox_inches='tight')